In [2]:
from keras.layers import Dense,MaxPooling2D,Flatten,Input,Conv2D,Concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import os
import numpy as np 

In [3]:
batch_size=40

In [4]:
datagen=ImageDataGenerator(rescale=1./255,rotation_range=40,zoom_range=0.2,width_shift_range=0.2)
# imageGenerator=datagen.flow_from_directory('/content/sample_data/data/train',batch_size=batch_size,target_size=(200,200),class_mode='binary')

In [5]:
from keras.applications import resnet


In [6]:
base_con=resnet.ResNet50(weights='imagenet',include_top=False,input_shape=(200,200,3))

In [7]:
def extract_feature(directory,sample_count):
  feature=np.zeros(shape=(sample_count,7,7,2048))
  labels=np.zeros(shape=(sample_count))
  imageGenerator=datagen.flow_from_directory(directory,
                                             batch_size=batch_size,target_size=(200,200),class_mode='binary')
  i=0
  for image , label in imageGenerator:
    F=base_con.predict(image)

    feature[i*batch_size:(i+1)*batch_size]=F
    labels[i*batch_size:(i+1)*batch_size]=label
    i+=1
    if i*batch_size>sample_count:
      break
    return feature, labels

In [8]:
train_dir='/content/sample_data/data/train/'

In [9]:
train_feature,train_label=extract_feature(train_dir,4640)

Found 4640 images belonging to 3 classes.


In [10]:
dim=train_feature.shape[1]*train_feature.shape[2]*train_feature.shape[3]
train_feature=np.reshape(train_feature,newshape=(4640,dim))

In [11]:
train_feature.shape

(4640, 100352)

In [12]:
import cv2
def thresh(image):
    image=np.reshape(image,newshape=(200,200,3))
    image=np.asarray(image)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    _,thresh=cv2.threshold(image,35,255,cv2.THRESH_BINARY)
    return thresh

In [13]:
FGenerator=datagen.flow_from_directory(train_dir,target_size=(200,200),batch_size=1,class_mode='binary')

Found 4640 images belonging to 3 classes.


In [15]:
count=0
feature_1=np.zeros(shape=(4640,200,200))
for image , label in FGenerator:
    f=thresh(image)
    feature_1[count:(count+1)]=f
    count+=1
    if count ==130:
        break

In [16]:
feature_1=np.reshape(feature_1,newshape=(4640,200*200))
feature_1.shape

(4640, 40000)

In [17]:
from keras.layers import concatenate

In [20]:
image=Input(shape=(100352))
F=Input(shape=40000)
sum=concatenate([image,F])

dense_2=Dense(120,activation='relu')(sum)
dense_3=Dense(100,activation='relu')(dense_2)
dense_4=Dense(50,activation='relu')(dense_3)
output=Dense(1,activation='sigmoid')(dense_4)

model=Model([image,F],output)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [21]:
history=model.fit([train_feature,feature_1],train_label,batch_size=batch_size,validation_split=0.2,epochs=30)

Epoch 1/30
93/93 [==============================] - 4s 28ms/step - loss: -11.6516 - acc: 0.9849 - val_loss: 0.3823 - val_acc: 1.0000
Epoch 2/30
93/93 [==============================] - 2s 20ms/step - loss: -382.8847 - acc: 0.9954 - val_loss: 0.0190 - val_acc: 1.0000
Epoch 3/30
93/93 [==============================] - 2s 20ms/step - loss: -2921.2986 - acc: 0.9954 - val_loss: 7.2781e-04 - val_acc: 1.0000
Epoch 4/30
93/93 [==============================] - 2s 20ms/step - loss: -12059.4658 - acc: 0.9954 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 5/30
93/93 [==============================] - 2s 20ms/step - loss: -31467.9980 - acc: 0.9954 - val_loss: 0.0076 - val_acc: 1.0000
Epoch 6/30
93/93 [==============================] - 2s 21ms/step - loss: -67830.5703 - acc: 0.9954 - val_loss: 2.4685e-06 - val_acc: 1.0000
Epoch 7/30
93/93 [==============================] - 2s 20ms/step - loss: -132371.9844 - acc: 0.9954 - val_loss: 7.9794e-08 - val_acc: 1.0000
Epoch 8/30
93/93 [=======================

In [22]:
model.save('/content/sample_data/my_model_1.h5',)